In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('../input/515k-hotel-reviews-data-in-europe/Hotel_Reviews.csv'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import numpy as np #array
import pandas as pd #data frame
import matplotlib.pyplot as plt #affichage
import seaborn as sns #affichage
import re #gerer les chaine de caractére en python
import nltk #traitement automatique des langues
import string
from nltk.corpus import stopwords #stop words . , !
from nltk.classify import SklearnClassifier
from nltk.tokenize import sent_tokenize, word_tokenize
import time
import nltk.classify.util
from nltk.classify import NaiveBayesClassifier 
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier 

In [ ]:
df=pd.read_csv('../input/515k-hotel-reviews-data-in-europe/Hotel_Reviews.csv',encoding='utf-8')

In [ ]:
display(df.shape)
display(df.describe())
display(df.describe(include=['O']))

In [ ]:
df.keys()

In [ ]:
# sélectionner les colonnes pertinentes pour l'analyse des sentiments des avis
review_data = df[['Hotel_Name', 'Positive_Review', 'Negative_Review', 'Average_Score', 'Reviewer_Score']].copy()
review_data.head()

In [ ]:
# Concaténation des avis positive et négative
review_data['reviews'] = review_data[['Positive_Review', 'Negative_Review']].apply(lambda x: ' '.join(x), axis = 1)
#review_data.head()
print(review_data.loc[3, 'reviews'])

In [ ]:
review_data

In [ ]:
# vérification de la distribution du score de révision
print(f"Review score given by Customer: {review_data.Reviewer_Score.value_counts()}")
review_data.Reviewer_Score.value_counts().plot(kind='bar', title='Count of Reviews', figsize = (15, 4))

In [ ]:
# Autre representation de score de révision à l'entier le plus proche
review_data['round_review_score'] = review_data.Reviewer_Score.apply(lambda x: np.ceil(x))
review_data.round_review_score.value_counts().plot(kind = 'bar', figsize=(16, 8), title = 'distribution of reviews')

In [ ]:
# les pays utilisés
df['Country']=df['Hotel_Address'].apply(lambda x: x.split()[-1])
df.Country.unique()

In [ ]:
# les pays qui donne le plus d'avis
# pour obtenir la nationalité la plus courante des avis pour chaque hôtel
most_national=df[['Hotel_Name','Hotel_Address','Reviewer_Nationality']].groupby(['Hotel_Name','Hotel_Address']).agg(lambda x:x.value_counts().index[0])

In [ ]:
# Dans le cas où nous voulons utiliser les tags, ici je nettoie le format
# et créer une liste à partir de tags, les a classés par occurrence
tags_rank=pd.Series(re.findall(r'[\']\s([\w\s]+)\s[\']',''.join(df.Tags))).value_counts()

In [ ]:
# Pour certains jours de raison où pas des nombres mais des chaînes donc ..
# Jour correct de l'examen en entier
df['days_since_review']=pd.to_numeric(df['days_since_review'].str.replace(r'[a-z]+', ""))

In [ ]:
most_national

In [ ]:
# Permet d'isoler les avis qui contiennent des informations en négatif / positif
# Pour plus de simplicité, je commence à obtenir les avis mal notés et très bien notés
# pour alimenter plus tard le classificateur.
# C'est ainsi qu'il / elle comprendra à quoi ressemble une mauvaise ou une bonne critique
# DataFrames avec avis négatifs et avis positifs donnés
neg_rev=df[df.Negative_Review!='No Negative'].reset_index().drop('index',1)
pos_rev=df[df.Positive_Review!='No Positive'].reset_index().drop('index',1)
neg_rev = neg_rev[neg_rev['Reviewer_Score']<5].reset_index().drop('index',1)
pos_rev = pos_rev[pos_rev['Reviewer_Score']>8].reset_index().drop('index',1)

In [ ]:
#import nltk #
#nltk.download('stopwords') #chaque dict elle a les mots qui on peut le filtrer
#from nltk.corpus import stopwords 
#from sklearn.feature_extraction.text import TfidfVectorizer

#processed_features = []
#for sentence in range(0, len(features)):
#    # Remove all the special characters   
#    processed_feature = re.sub(r'\W', ' ', str(features[sentence]))
#    processed_feature= re.sub(r'\s+[a-zA-Z]\s+', ' ', processed_feature)   #suppprimer les caractére 
#    # Remove single characters from the start 
#    processed_feature = re.sub(r'\^[a-zA-Z]\s+', ' ', processed_feature) 
#    # Substituting multiple spaces with single space 
#    processed_feature = re.sub(r'\s+', ' ', processed_feature, flags=re.I) 
#    # Removing prefixed 'b'  
#    processed_feature = re.sub(r'^b\s+', '', processed_feature)   
#    # Converting to Lowercase 
#    processed_feature = processed_feature.lower()  
#    processed_features.append(processed_feature) 
#vectorizer = TfidfVectorizer (max_features=2500, min_df=7, max_df=0.8, stop_words=stopwords.words('english')) 
#processed_features = vectorizer.fit_transform(processed_features).toarray()


In [ ]:
# Takes review and gives back the clean list of words

#TokTok faster than word_tokenize
from nltk.tokenize import ToktokTokenizer
toktok = ToktokTokenizer()

# Stopwords, numbers and punctuation to remove
remove_punct_and_digits = dict([(ord(punct), ' ') for punct in string.punctuation + string.digits])
stopWords = set(stopwords.words('french'))


def word_cleaner(data):
    cleaned_word = data.lower().translate(remove_punct_and_digits)
    words = word_tokenize(cleaned_word)
    words = [toktok.tokenize(sent) for sent in sent_tokenize(cleaned_word)]
    wordsFiltered = []
    if not words:
        pass
    else:
        for w in words[0]:
            if w not in stopWords:
                wordsFiltered.append(w)
                end=time.time()
    return wordsFiltered


# Example
wordsFiltered = word_cleaner(neg_rev.Negative_Review[1])
print(neg_rev.Negative_Review[1])
print(wordsFiltered)

In [ ]:
# Select features from original dataset to form a new dataframe 
df1 = review_data[['reviews']]
# For each row, combine all the columns into one column 
df2 = df1.apply(lambda x: ','.join(x.astype(str)), axis=1)
# Store them in a pandas dataframe 
df_clean = pd.DataFrame({'clean': df2}) 
# Create the list of list format of the custom corpus for gensim modeling 
sent = [row.split(',') for row in df_clean['clean']] 
# show the example of list of list format of the custom corpus for gensim modeling
sent[:2]

In [ ]:
#word embeding(word2vec)
from gensim.models import Word2Vec 
#model word embeding
model = Word2Vec(min_count=1,size= 50,workers=3, window =3, sg = 1)


In [ ]:
#test de vocabulary
model['damage']

In [ ]:
neg_red=neg_rev[:50000].copy()
pos_red=pos_rev[:50000].copy()

In [ ]:
# Créer un ensemble lié aux avis positifs et négatifs
def word_feats(words):
    return dict([(word, True) for word in words])
neg_set=[(word_feats(word_feats(word_cleaner(neg_red.loc[i,'Negative_Review']))), 0) for i in range(len(neg_red))]
pos_set=[(word_feats(word_feats(word_cleaner(pos_red.loc[i,'Positive_Review']))), 1) for i in range(len(pos_red))]

In [ ]:
# Enfin, une machine learning test!
# Train model et utilisé CV pour test accuracy

negcutoff = int(len(neg_set)*3/4)
poscutoff = int(len(pos_set)*3/4)
 
trainfeats = neg_set[:negcutoff] + pos_set[:poscutoff]
testfeats = neg_set[negcutoff:] + pos_set[poscutoff:]
print(len(trainfeats), len(testfeats))
 
classifier = NaiveBayesClassifier.train(trainfeats)
print( 'accuracy:', nltk.classify.util.accuracy(classifier, testfeats))